In [1]:
!pip install fastapi uvicorn pyngrok nest_asyncio
from pyngrok import ngrok
ngrok.set_auth_token("32ZUB4IjFR36X0224ddc8OKppC2_83JHp1GbLJhAaa9uhV9vh")

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import nest_asyncio
from pyngrok import ngrok
import torch
import torch.nn as nn
import torch.optim as optim

# مدل
class prediction_net(nn.Module):
    def __init__(self, w, n_features_input, lr=0.01):
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.prediction_net = nn.Sequential(
            nn.Linear(w + n_features_input, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 6),
            nn.LeakyReLU(),
            nn.Linear(6, 1)
        ).to(self.device)

        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(), lr=lr)

    def forward(self, combined_embedded, label=None, status='test'):
        combined_embedded = torch.tensor(combined_embedded, dtype=torch.float, device=self.device)
        combined_embedded.requires_grad_(True)

        if status == 'train':
            label = torch.tensor(label, dtype=torch.float, device=self.device)
            self.optimizer.zero_grad()
            output = self.prediction_net(combined_embedded)
            loss = self.loss_fn(output, label)
            loss.backward()
            input_grad = combined_embedded.grad.detach().cpu().tolist()
            self.optimizer.step()
            result = {'grad' : input_grad}
            return result
        else:  # test
            output = self.prediction_net(combined_embedded)
            result = output.detach().cpu().tolist()
            result = {'prediction': result}
            return result


# مدل پایدنتیک برای درخواست
class Data(BaseModel):
    prediction_iput: list
    label: list = None
    status: str


# FastAPI
app = FastAPI()

# مدل یک بار ساخته شود
model = prediction_net(w=3, n_features_input=4, lr=0.01)

@app.post("/prediction")
def prediction(data: Data):
    combined_embedded = data.prediction_iput
    label = data.label
    status = data.status
    result = model(combined_embedded, label, status)
    return result


# ngrok و uvicorn
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

nest_asyncio.apply()
uvicorn.run(app, port=8000)


INFO:     Started server process [325]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://701c61029189.ngrok-free.app
